In [2]:
"""
Needleman-Wunsch-Algorithmを用いたシーケンスアライメント
2つの文字列の一致する数が最大に，不一致の数が最小になるようにスコアを元に配列の並び替えを行う
2つの文字列は同じ配列長になる．適宜ダミー文字"-"が挿入される．
"""

from __future__ import print_function, division
import numpy as np

pt ={'match': 2, 'mismatch': -1, 'gap': -1}

#Needleman-Wunsch-Algorithmのscoreを計算する
def mch(alpha, beta):

    if alpha == beta:
        return pt['match']
    elif alpha == '-' or beta == '-':
        return pt['gap']
    else:
        return pt['mismatch']

    
#2つの文章のシーケンスアライメントを行う
def alignment(s1, s2):
    m, n = len(s1), len(s2)
    score = np.zeros((m+1, n+1))
    
    #Initialization
    for i in range(m+1):
        score[i][0] = pt['gap'] * i
    for j in range(n+1):
        score[0][j] = pt['gap'] * j
    
    #Fill
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            diag = score[i-1][j-1] + mch(s1[i-1], s2[j-1])
            #print(s1[i-1], s2[j-1])
            delete = score[i-1][j] + pt['gap']
            insert = score[i][j-1] + pt['gap']
            score[i][j] = max(diag, delete, insert)

    align1 = []
    align2 = []
    i,j = m,n
    
    #Traceback
    while i > 0 and j > 0:
        score_current = score[i][j]
        score_diag = score[i-1][j-1]
        score_left = score[i][j-1]
        score_up = score[i-1][j]
 
        if score_current == score_diag + mch(s1[i-1], s2[j-1]):
            a1,a2 = s1[i-1],s2[j-1]    
            i,j = i-1,j-1
        elif score_current == score_up + pt['gap']:
            a1,a2 = s1[i-1],'-'  
            i -= 1
        elif score_current == score_left + pt['gap']:
            a1,a2 = '-',s2[j-1]
            j -= 1
        align1.append(a1)
        align2.append(a2)
        
    align1 = align1[::-1]
    align2 = align2[::-1]
    seqN = len(align1)
    sym = ''
    seq_score = 0
    ident = 0

    for i in range(seqN):
        a1 = align1[i]
        a2 = align2[i]
        if a1 == a2:
            sym += a1
            ident += 1
            seq_score += mch(a1, a2)
    
        else: 
            seq_score += mch(a1, a2)
            sym += '-'
        
    ident = ident/seqN * 100
    align1="".join(align1)
    align2="".join(align2)
    
    return ident, seq_score, align1, align2, sym


"""
形態素解析器Mecabを用いて，入力された文字列の形態素解析を行う(単語に分割する)
"""

import MeCab
mecab = MeCab.Tagger ()
mecab.parse('')#文字列がGCされるのを防ぐ

#形態素解析(特定の品詞のみ)
def keitaiso(text):
    node = mecab.parseToNode(text)
    word=[]

    while node:
        feats = node.feature.split(',')
        try:
            word.append(node.surface)  #単語を取得
        except:
            print("err: " + str(node.surface))
        node = node.next  #次の単語に進める
    return word
        

    
            
if __name__ == '__main__':
    
    #比較を行う２つの文字列の入力
    text1 = "私は自然言語の処理はとても難しいと思う．"
    text2 = "僕はとても自然言語処理は難しいと思います．"
    print("text1 : " + text1)
    print("text2 : " + text2)
    print()
    
    
    #二つの文字列に形態素解析を行い，単語に分割
    tokenized_text1=keitaiso(text1)
    tokenized_text2=keitaiso(text2)
    print("text1(形態素解析) : " + str(tokenized_text1))
    print("text2(形態素解析) : " + str(tokenized_text2))
    print()
    
    
    #形態素解析を行なった2つのテキストに対して，シーケンスアライメントを行い，最適なアライメントを取得
    ident, score, align1, align2, sym = alignment(tokenized_text1, tokenized_text2)
    print("text1(アライメント) : " + align1)
    print("text2(アライメント) : " + align2)
    print()
    print("共通部分 : " + sym)
    print("一致率 : " + str(ident) + " %")
    print("Needleman-Wunsch-Algorithmのスコア : " + str(score))
    

text1 : 私は自然言語の処理はとても難しいと思う．
text2 : 僕はとても自然言語処理は難しいと思います．

text1(形態素解析) : ['', '私', 'は', '自然', '言語', 'の', '処理', 'は', 'とても', '難しい', 'と', '思う', '．', '']
text2(形態素解析) : ['', '僕', 'は', 'とても', '自然', '言語', '処理', 'は', '難しい', 'と', '思い', 'ます', '．', '']

text1(アライメント) : 私は-自然言語の処理はとても難しいと-思う．
text2(アライメント) : 僕はとても自然言語-処理は-難しいと思います．

共通部分 : -は-自然言語-処理は-難しいと--．
一致率 : 62.5 %
Needleman-Wunsch-Algorithmのスコア : 14
